In [1]:
import os,sys,json,re
sys.path.insert(0, '../../scripts/utils')
from prettyjson import prettyjson
import ase
from ase.io import read
from ase.visualize import view
import numpy as np
import ubjson, json

In [4]:
!ls repo-fit-bulk/

README                                 mebox-minimal-oos-T188-pbe-b1b.xyz
dispts_quip_params.xml                 mebox-minimal-oos-T188-pbe-mbdint.xyz
fit_mbd_soap.sh                        mebox-minimal-oos-T188-pbe0-b1b.xyz
fit_soap_box.sh                        mebox-minimal-oos-T188-pbe0-mbdint.xyz
mebox-minimal-oos-T110-pbe-b1b.xyz     mebox-minimal-pbe-b1b.xyz
mebox-minimal-oos-T110-pbe-mbdint.xyz  mebox-minimal-pbe-mbdint.xyz
mebox-minimal-oos-T110-pbe0-b1b.xyz    mebox-minimal-pbe0-b1b.xyz
mebox-minimal-oos-T110-pbe0-mbdint.xyz mebox-minimal-pbe0-pbeotf-mbdint.xyz


In [49]:
fn = '/Users/iMac/Downloads/input.data'
with open(fn, 'r') as f:
    data = f.read().encode('unicode-escape').decode()

In [111]:
patterns = {
    'struture':r'(?<=begin).*?(?=end)',
    'lattice':r'(?<=lattice).*?(?=\\n)',
    'atom':r'(?<=atom).*?(?=\\n)',
   'float':r"[-+]?[0-9]*\.?[0-9]+",
    'energy':r"energy\s+([-+]?[0-9]*\.?[0-9]+)\s+\\n",
   'species':r"[BCFHIKNOPSUVWY]|A[cglmrstu]|B[aehikr]|C[adeflmnorsu]|D[bsy]|E[rsu]|F[elmr]|G[ade]|H[efgos]|I[nr]|Kr|L[airuv]|M[dgnot]|N[abdeiop]|Os|P[abdmortu]|R[abefghnu]|S[bcegimnr]|T[abcehilm]|Uu[opst]|Xe|Yb|Z[nr]",
           }
frames = []
for res in results:
    ddd = re.findall(patterns['lattice'] , res)
    lat = []
    for d in ddd:
        lat.append(list(map(float,re.findall(patterns['float'] , d))))
    
    ddd = re.findall(patterns['atom'] , res)
    pos,sym,force = [],[],[]
    for d in ddd:
        aa = d.split()
        pos.append(list(map(float,aa[:3])))
        force.append(list(map(float,aa[-3:])))
        aa = aa[3]
        sym.append(aa)
    energy = float(re.findall(patterns['energy'] , res)[0])
    frame = ase.Atoms(positions=pos,symbols=sym,cell=lat,pbc=True)
    frame.info['energy'] = energy
    frame.set_array('forces',np.array(force))
    frame.wrap(eps=1e-11)
    frames.append(frame)

In [154]:
frame.info

{'energy': -1272.64893}

In [2]:
list(map("{}".format,range(1, 10+1)))

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

In [4]:
aa = 10
str(aa)

'10'

In [9]:
fn_out = '../methane_sulfonic.ubjson'
tofile(fn_out,frames)

NameError: name 'frames' is not defined

In [168]:
frames = fromfile(fn_out)

In [169]:
len(frames)

4141

In [170]:
view(frames)

In [2]:
def tofile(fn,frames):
    import numpy as np
    keys = ['positions','cell','numbers','pbc']
    data = {}
    for ii,frame in zip(range(1, len(frames)+1), frames):
        aa = dict(positions=frame.get_positions().tolist(),cell=frame.get_cell().tolist(),
                numbers=frame.get_atomic_numbers().tolist(),pbc=frame.get_pbc().tolist())
        aa['info'] = {}
        for k,v in  frame.info.items():
            if isinstance(v, np.integer):
                aa['info'][k] = int(v)
            elif isinstance(v, np.bool_):
                aa['info'][k] = bool(v)
            elif isinstance(v, np.float):
                aa['info'][k] = float(v)
            elif hasattr(v, 'tolist'):
                aa['info'][k] = v.tolist()
            else:
                aa['info'][k] = v
        aa['arrays'] = {}
        for k,v in frame.arrays.items():
            if k not in keys:
                aa['arrays'][k] = v.tolist()
        data[str(ii)] = aa
    data['ids'] = list(range(1, len(frames)+1))
    data['nextid'] = len(frames)+1
    _, extension = os.path.splitext(fn)
    if extension == '.json':
        with open(fn, 'w') as f:
            # json.dump(data, f)
            data_pretty = prettyjson(data,indent=2, maxlinelength=80)
            f.write(data_pretty)
    elif extension == '.ubjson':
        import ubjson
        with open(fn, 'wb') as f:
            ubjson.dump(data, f,no_float32=False)
            
def fromfile(fn):
    _, extension = os.path.splitext(fn)
    if extension == '.json':
        frames = read(fn, ':')
    elif extension == '.ubjson':
        import ubjson
        with open(fn, 'rb') as f:
            data = ubjson.load(f)
        frames = []
        for idx in data['ids']:
            ff = data['{}'.format(idx)]
            frame = ase.Atoms(positions=ff['positions'], cell=ff['cell'], 
                              numbers=ff['numbers'], pbc=ff['pbc'])
            if 'info' in ff:
                frame.info = ff['info']
            if 'arrays' in ff:
                for k,v in ff['arrays'].items():
                    frame.set_array(k, np.array(v))
            frames.append(frame)
    return frames

In [26]:
fns = {'molecular_crystals':'CSD1000-r.xyz',
       'silicon_bulk':'gp_iter6_sparse9k.xml.xyz',
       'qm9':'qm9_5000.xyz',
       'methane_liquid':'repo-fit-bulk/mebox-minimal-pbe0-b1b.xyz'}
# fns = {'molecular_crystals':'CSD1000-r.xyz'}
for k,fn in fns.items():
    frames = read(fn,':')
    for frame in frames:
        if k == 'qm9':
            frame.set_pbc(False)
            frame.set_cell(np.eye(3)*10)
            frame.center(about=[5,5,5])
        frame.wrap(eps=1e-11)
            
    print(k,len(frames))
    fn_out = '../'+k+'.json'
    tofile(fn_out,frames)

molecular_crystals 1000
silicon_bulk 2475
qm9 5000
methane_liquid 258


In [9]:
from ase.neighborlist import neighbor_list
path = '../'
fns = ['qm9.json', 'molecular_crystals.json', 'silicon_bulk.json', 
       'methane_liquid.json', 'methane_sulfonic.ubjson']
sp = {
    # 'qm9.json' : {'st':0,'nd':500, 'c':4, 'o':'qm9_0_500.json'}, 
    'molecular_crystals.json' : {'lg':30, 'c':4, 'o':'molecular_crystals_50.json'}, 
    # 'silicon_bulk.json' : {'st':600,'nd':700, 'c':4, 'o':'silicon_bulk_600_700.json'}, 
    # 'methane_liquid.json' : {'st':100,'nd':150, 'c':4, 'o':'methane_liquid_100_150.json'}, 
    # 'methane_sulfonic.ubjson' : {'st':0,'nd':150, 'c':4, 'o':'methane_sulfonic_0_150.json'}
}


for fn in sp:
    frames = fromfile(path+fn)
    s = sp[fn]
    ffs = frames
    nat = 0
    nneigh = 0
    sps = []
    sel = []
    ii = 0
    for ff in ffs:
        if ii >= s['lg']:
            break
        if np.product(ff.get_pbc()):
            cl = ff.get_cell_lengths_and_angles()[:3]
            if np.any(cl < 2 * s['c']):
                continue
            else:
                sel.append(ff)
                ii += 1
        i,j = ase.neighborlist.neighbor_list('ij', ff, s['c']) 
        nat += len(np.unique(i))
        nneigh += len(j)
        sps.extend(ff.get_atomic_numbers())
    print(fn, nat, nneigh, np.unique(sps))
    # tofile(path+s['o'], sel)

KeyError: 'lg'

In [36]:
view(sel)

In [33]:
from rascal.neighbourlist.base import _neighbourlist_list
_neighbourlist_list

['centers',
 'neighbourlist',
 'centercontribution',
 'strict',
 'maxorder',
 'halflist',
 'fulllist']

In [6]:
frames = fromfile('../molecular_crystals_50.json')
view(frames)

In [4]:
len(frames)

1000

In [31]:
!ls ../

methane_liquid.json	 qm9_0_500.json  README.md
methane_sulfonic.ubjson  qm9.json	 silicon_bulk_600_700.json
molecular_crystals.json  raw_data	 silicon_bulk.json
